In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os 
os.chdir("/content/drive/My Drive/Gutenberg_books")


In [5]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
# Read the csv file into pandas dataframe
data = pd.read_csv("master996.csv",encoding = "ISO-8859-1")

#Replace book id with the actual name present in html so as to iterate through html files
data['book_id'] = data['book_id'].str.replace('.epub', '-content.html', case = False)
book_id_array = data['book_id'].to_numpy()
#print(book_id_array[0])

genre = ['guten_genre']

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder

# Read the csv file into pandas dataframe
data = pd.read_csv("master996.csv",encoding = "ISO-8859-1")

#Replace book id with the actual name present in html so as to iterate through html files
data['book_id'] = data['book_id'].str.replace('.epub', '-content.html', case = False)
book_id_array = data['book_id'].to_numpy()
#print(book_id_array[0])

genre = ['guten_genre']

#Convert each genre of data dataframe into numbers
for x in genre:
    le = LabelEncoder()
    le.fit(list(data[x].values))
    data[x] = le.transform(list(data[x]))
print('******************Dataframe after converting Genre as numbers**********************')
print(data)

wordArray = []
index = 0

#Iterate through each .html files and read the contents and append to our exisitng dataframe
while index < len(book_id_array):
	#Open the HTML file abd read the contents for tokenizing
	#Replace the path with the folder where html files are present
	#HTMLfile = open('E:\\OVGU\\Sem2_Summer2020\\Subjects\\ATML\\Semester_Project\\starter\\Gutenberg_English_Fiction_1k\\Gutenberg_19th_century_English_Fiction\\'+book_id_array[index],'r',encoding='utf-8')
	HTMLfile = open('/content/drive/My Drive/Gutenberg_books/Gutenberg_19th_century_English_Fiction/'+book_id_array[index],'r',encoding='utf-8')

	content = HTMLfile.read()
	r = re.compile('<.*?>')
	content_removed_tags = re.sub(r, '', content)

	#Pick pre-defineds Stop words from nltk lib
	stop_words = list(stopwords.words('english'))

	# Tokenize the text
	tokens = word_tokenize(content_removed_tags)
	# Convert into lower case
	tokens = [w.lower() for w in tokens]
	print('Document ',index,':')
	print('After Tokenizing: ',len(tokens))

	#Remove non alphabetic words/Removing punctuations
	words = [word for word in tokens if word.isalpha()]
	print('After Removing Punctuations: ',len(words))
	#Removing Stopwords
	words = [w for w in words if not w in stop_words]
	print('After Removing Stopwords: ',len(words))
	
	wordArray.insert(index,words)
	index = index+1








In [15]:
print(len(wordArray))

data['tokens'] = wordArray
#Convert the list into strings, column should have tokens seperated by space
data['list_string'] = data['tokens'].apply(lambda x: ' '.join(map(str, x)))

print('******************Dataframe after appending Tokens**********************')
#Dataframe after appending the tokens of respective books into data['list_string']
print(data)

vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(data['list_string'])
print('************************Vectors Shape*******************************')
print(vectors.shape)

X_train, X_test, y_train, y_test = train_test_split(vectors, data['guten_genre'], test_size=0.3)
print('*******************Train and Test Set Size**************************')
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

996
******************Dataframe after appending Tokens**********************
                                             Book_Name  ...                                        list_string
0    The Mystery of the Boule Cabinet: A Detective ...  ...  detective story hello said took receiver desk ...
1                                            The Pupil  ...  edition first published text follows definitiv...
2                                       At Love's Cost  ...  moment never fully realised great ass man thin...
3                               The Heart of the Range  ...  rider golden bar hidden trails lynch lawyers o...
4                          The Worshipper of the Image  ...  worshipper image evening wood still dreaming b...
..                                                 ...  ...                                                ...
991                                  David Copperfield  ...  find easy get sufficiently far away book first...
992                                

In [16]:
clf = MultinomialNB(alpha=.45)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print('*******************F1 Score and Accuracy*****************************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='macro'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='macro'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='macro'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))

*******************F1 Score and Accuracy*****************************
F1 Score:  0.36980426183012394
Precision Score:  0.3475478636721743
Recall Score:  0.4063309571784148
Accuracy:  0.8294314381270903


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
